In [3]:
import collections
import pathlib
import re
import string

import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras import utils
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

#import tensorflow_datasets as tfds
#import tensorflow_text as tf_text

In [4]:
data_url = 'https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz'
dataset = utils.get_file('stack_overflow_16k.tar.gz',data_url,untar=True,cache_dir='stack_overflow',cache_subdir='')
dataset_dir = pathlib.Path(dataset).parent

6053888/6053168 [==============================] - 2s 0us/step


In [5]:
list(dataset_dir.iterdir())

[WindowsPath('/tmp/.keras/README.md'),
 WindowsPath('/tmp/.keras/stack_overflow_16k.tar.gz.tar.gz'),
 WindowsPath('/tmp/.keras/test'),
 WindowsPath('/tmp/.keras/train')]

In [20]:
train_dir = dataset_dir/'train'
test_dir = dataset_dir/'test'

list(train_dir.iterdir())

[WindowsPath('/tmp/.keras/train/csharp'),
 WindowsPath('/tmp/.keras/train/java'),
 WindowsPath('/tmp/.keras/train/javascript'),
 WindowsPath('/tmp/.keras/train/python')]

In [7]:
sample_file = train_dir/'python/1755.txt'
with open(sample_file) as f:
  print(f.read())

why does this blank program print true x=true.def stupid():.    x=false.stupid().print x



In [21]:
batch_size = 32
seed = 42

raw_train_ds = preprocessing.text_dataset_from_directory(train_dir,batch_size=batch_size,validation_split=0.2,
                                                         subset='training',seed=seed)
raw_val_ds = preprocessing.text_dataset_from_directory(train_dir,batch_size=batch_size,validation_split=0.2,
                                                       subset='validation',seed=seed)

raw_test_ds = preprocessing.text_dataset_from_directory(test_dir, batch_size=batch_size)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.
Found 8000 files belonging to 4 classes.
Using 1600 files for validation.
Found 8000 files belonging to 4 classes.


In [18]:
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(10):
    #print("\nQuestion: ", text_batch.numpy()[i])
    print("Label:", label_batch.numpy()[i])

Label: 3
Label: 3
Label: 2
Label: 0
Label: 0
Label: 0
Label: 2
Label: 0
Label: 2
Label: 1


In [16]:
for i, label in enumerate(raw_train_ds.class_names):
  print("Label", i, "corresponds to", label)

Label 0 corresponds to csharp
Label 1 corresponds to java
Label 2 corresponds to javascript
Label 3 corresponds to python


In [22]:
VOCAB_SIZE = 10000
MAX_SEQUENCE_LENGTH = 250

binary_vectorize_layer = TextVectorization(max_tokens=VOCAB_SIZE,output_mode='binary')

int_vectorize_layer = TextVectorization(max_tokens=VOCAB_SIZE,output_mode='int',
                                        output_sequence_length=MAX_SEQUENCE_LENGTH)

In [23]:
# Make a text-only dataset (without labels), then call adapt
train_text = raw_train_ds.map(lambda text, labels: text)
binary_vectorize_layer.adapt(train_text)
int_vectorize_layer.adapt(train_text)

In [24]:
def binary_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return binary_vectorize_layer(text), label

def int_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return int_vectorize_layer(text), label

In [25]:
# Retrieve a batch (of 32 reviews and labels) from the dataset
text_batch, label_batch = next(iter(raw_train_ds))
first_question, first_label = text_batch[0], label_batch[0]
print("Question", first_question)
print("Label", first_label)

Question tf.Tensor(b'"set blank to quit on exception? i\'m using blank 3..i\'ve been looking around for an answer to this, but i haven\'t found it yet. basically, i\'m running several blank scripts into a game engine, and each script has its own entry point...i\'d rather not add try: except blocks through all of my code, so i was wondering if it\'s at all possible to tell blank to quit (or perhaps assign a custom function to that ""callback"") on finding its first error, regardless of where or what it found? ..currently, the game engine will continue after finding and hitting an error, making it more difficult than necessary to diagnose issues since running into one error may make a subsequent script not work (as it relies on variables that the error-ing script set, for example). any ideas? ..i know that i could redirect the console to a file to allow for easier scrolling, but just capturing the first error and stopping the game prematurely would be really useful...okay, a couple of ex

In [27]:
print("'binary' vectorized question:", binary_vectorize_text(first_question, first_label)[0])
print("'int' vectorized question:",int_vectorize_text(first_question, first_label)[0])

'binary' vectorized question: tf.Tensor([[1. 1. 1. ... 0. 0. 0.]], shape=(1, 10000), dtype=float32)
'int' vectorized question: tf.Tensor(
[[ 107   16    4 1138   38  184   52   47   16    1  215  416  518   12
    32  181    4   13   26    3 1414  227   11  894  627   52  309  718
    16 1272  100    5  244 1826    8  119  223   95   96  657  858    1
   797   20  125  117  559 2280  194   73    9   23   30   49    3  114
   566   10   96   59   73  206    4  412   16    4 1138   45 1669  600
     5  696   37    4   14  850   38  966   96   98   65 2373    9  132
    45   55   11  227  403    2  244 1826   72  534  156  966    8 7799
    32   65  469   11  182 2332  198 1460    4    1 1093  447  309  100
    71   65  454  109    5 3554  223   20  139   36   11 9038   38  233
    14    2    1  223  107   12  137   76  778    3  103   14    3  177
  1852    2  332    4    5   39    4  787   12 1319 3573   26  106    1
     2   98   65    8 2900    2  244    1   69   33  336    1    5 175

In [28]:
print("107 ---> ", int_vectorize_layer.get_vocabulary()[107])
print("16 ---> ", int_vectorize_layer.get_vocabulary()[16])
print("Vocabulary size: {}".format(len(int_vectorize_layer.get_vocabulary())))

107 --->  set
16 --->  blank
Vocabulary size: 10000


In [29]:
binary_train_ds = raw_train_ds.map(binary_vectorize_text)
binary_val_ds = raw_val_ds.map(binary_vectorize_text)
binary_test_ds = raw_test_ds.map(binary_vectorize_text)

int_train_ds = raw_train_ds.map(int_vectorize_text)
int_val_ds = raw_val_ds.map(int_vectorize_text)
int_test_ds = raw_test_ds.map(int_vectorize_text)

In [30]:
AUTOTUNE = tf.data.AUTOTUNE

def configure_dataset(dataset):
  return dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [31]:
binary_train_ds = configure_dataset(binary_train_ds)
binary_val_ds = configure_dataset(binary_val_ds)
binary_test_ds = configure_dataset(binary_test_ds)

int_train_ds = configure_dataset(int_train_ds)
int_val_ds = configure_dataset(int_val_ds)
int_test_ds = configure_dataset(int_test_ds)

In [32]:
binary_model = tf.keras.Sequential([layers.Dense(4)])
binary_model.compile(loss=losses.SparseCategoricalCrossentropy(from_logits=True),optimizer='adam',
                     metrics=['accuracy'])
history = binary_model.fit(binary_train_ds, validation_data=binary_val_ds, epochs=10)

Epoch 1/10
200/200 [==============================] - 15s 71ms/step - loss: 1.2445 - accuracy: 0.5153 - val_loss: 0.9166 - val_accuracy: 0.7738
Epoch 2/10
200/200 [==============================] - 0s 2ms/step - loss: 0.8225 - accuracy: 0.8145 - val_loss: 0.7519 - val_accuracy: 0.7956
Epoch 3/10
200/200 [==============================] - 0s 2ms/step - loss: 0.6542 - accuracy: 0.8600 - val_loss: 0.6659 - val_accuracy: 0.8131
Epoch 4/10
200/200 [==============================] - 1s 3ms/step - loss: 0.5540 - accuracy: 0.8851 - val_loss: 0.6123 - val_accuracy: 0.8181
Epoch 5/10
200/200 [==============================] - 0s 2ms/step - loss: 0.4846 - accuracy: 0.8996 - val_loss: 0.5755 - val_accuracy: 0.8263
Epoch 6/10
200/200 [==============================] - 0s 2ms/step - loss: 0.4323 - accuracy: 0.9170 - val_loss: 0.5489 - val_accuracy: 0.8306
Epoch 7/10
200/200 [==============================] - 0s 2ms/step - loss: 0.3908 - accuracy: 0.9266 - val_loss: 0.5288 - val_accuracy: 0.8338
Epoc

In [33]:
binary_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 40004     
Total params: 40,004
Trainable params: 40,004
Non-trainable params: 0
_________________________________________________________________


In [35]:
def create_model(vocab_size, num_labels):
  model = tf.keras.Sequential([
      layers.Embedding(vocab_size, 64, mask_zero=True),
      layers.Conv1D(64, 5, padding="valid", activation="relu", strides=2),
      layers.GlobalMaxPooling1D(),
      layers.Dense(num_labels)
  ])
  return model

In [36]:
# vocab_size is VOCAB_SIZE + 1 since 0 is used additionally for padding.
int_model = create_model(vocab_size=VOCAB_SIZE + 1, num_labels=4)
int_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])
history = int_model.fit(int_train_ds, validation_data=int_val_ds, epochs=5)

Epoch 1/5
200/200 [==============================] - 7s 33ms/step - loss: 1.3177 - accuracy: 0.3981 - val_loss: 0.8071 - val_accuracy: 0.6844
Epoch 2/5
200/200 [==============================] - 3s 16ms/step - loss: 0.7332 - accuracy: 0.7056 - val_loss: 0.5522 - val_accuracy: 0.7944
Epoch 3/5
200/200 [==============================] - 3s 16ms/step - loss: 0.4410 - accuracy: 0.8533 - val_loss: 0.4702 - val_accuracy: 0.8175
Epoch 4/5
200/200 [==============================] - 3s 16ms/step - loss: 0.2464 - accuracy: 0.9423 - val_loss: 0.4624 - val_accuracy: 0.8263
Epoch 5/5
200/200 [==============================] - 3s 16ms/step - loss: 0.1243 - accuracy: 0.9768 - val_loss: 0.4811 - val_accuracy: 0.8288


In [37]:
print("Linear model on binary vectorized data:")
print(binary_model.summary())

Linear model on binary vectorized data:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 40004     
Total params: 40,004
Trainable params: 40,004
Non-trainable params: 0
_________________________________________________________________
None


In [38]:
print("ConvNet model on int vectorized data:")
print(int_model.summary())

ConvNet model on int vectorized data:
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          640064    
_________________________________________________________________
conv1d (Conv1D)              (None, None, 64)          20544     
_________________________________________________________________
global_max_pooling1d (Global (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 260       
Total params: 660,868
Trainable params: 660,868
Non-trainable params: 0
_________________________________________________________________
None


In [39]:
binary_loss, binary_accuracy = binary_model.evaluate(binary_test_ds)
int_loss, int_accuracy = int_model.evaluate(int_test_ds)

print("Binary model accuracy: {:2.2%}".format(binary_accuracy))
print("Int model accuracy: {:2.2%}".format(int_accuracy))

250/250 [==============================] - 4s 15ms/step - loss: 0.5203 - accuracy: 0.8069
Binary model accuracy: 81.41%
Int model accuracy: 80.69%


# Exporte o modelo